In [ ]:
#@title Downloading helper files...<p>(Run this cell first!)\n",
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise13_data/cifar100/target_train_cifar100_indices.pt", "target_train_cifar100_indices.pt"),
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise13_data/cifar100/target_test_cifar100_indices.pt", "target_test_cifar100_indices.pt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise13_data/cifar100/shadow_cifar100_indices.pt", "shadow_cifar100_indices.pt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/priv-sec/privacy/master/exercise13_data/MIA.py", "MIA.py")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from IPython.display import clear_output
from tqdm import trange
import numpy as np

## Init dataset

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR100("./cifar100", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100("./cifar100", train=False, download=True, transform=transform)
dataset = train_dataset + test_dataset

target_train_dataset_indices = torch.load("target_train_cifar100_indices.pt")
target_train_dataset = torch.utils.data.Subset(dataset, target_train_dataset_indices)

target_test_dataset_indices = torch.load("target_test_cifar100_indices.pt")
target_test_dataset = torch.utils.data.Subset(dataset, target_test_dataset_indices)

## Create a neural network

In [ ]:
###################
#     EDIT ME     #
###################

class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (5, 5))
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6272, 128)
        self.fc2 = nn.Linear(128, 100)

    def forward(self, x):
        x = self.pool( F.relu( self.conv1(x) ) )
        x = torch.flatten(x, 1) 
        x = torch.tanh( self.fc1(x) )
        x = self.fc2(x)
        return x

## Train and test

In [ ]:
###################
#     EDIT ME     #
###################

def train(model, device, train_loader):
    """
    """

    model.train()

    optimizer = torch.optim.Adam(model.parameters())

    train_loss = 0
    correct = 0

    for data, target in train_loader:

        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        output = model(data)

        loss = F.cross_entropy(output, target)
        loss.backward()

        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)

    return train_loss, 100. * correct / len(train_loader.dataset)



def test(model, device, test_loader):
    """
    """

    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():

        for data, target in test_loader:

            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return test_loss, 100. * correct / len(test_loader.dataset)

## Main

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 32}     # <- EDIT ME
test_kwargs  = {'batch_size': 1000}   # <- EDIT ME
    
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_target_loader = torch.utils.data.DataLoader(target_train_dataset,**train_kwargs)
test_target_loader  = torch.utils.data.DataLoader(target_test_dataset, **test_kwargs)

model = CNN().to(device)

epochs = 40

In [ ]:
# TRAIN & SAVE...

for epoch in trange(1, epochs + 1):
    
    train_loss, train_acc = train(model, device, train_target_loader)
    
    clear_output(wait=True)
    test_loss, test_acc = test(model, device, test_target_loader)
    print("Train => Loss: {:.4f}, Accuracy: {:.2f}%".format(train_loss, train_acc))
    print("Test  => Loss: {:.4f}, Accuracy: {:.2f}%".format(test_loss, test_acc))
    
torch.save(model.state_dict(), "./target_model_cifar100.pt")

In [ ]:
# ...OR LOAD

model.load_state_dict(torch.load("./target_model_cifar100.pt"))  # Load pre-trained weights

## Test MIA attack

In [ ]:
#################
# DO NOT CHANGE #
#################

from MIA import get_mia_accuracy

shadow_dataset_indices = torch.load("shadow_cifar100_indices.pt")
shadow_dataset = torch.utils.data.Subset(dataset, shadow_dataset_indices)
attack_acc = get_mia_accuracy(CNN, model, epochs, device, (train_target_loader, test_target_loader), train_kwargs, train, shadow_dataset, 
                              15, load_shadow=False)
    
print("### MIA RESULTS ###")
print("Accuracy:", attack_acc)

## Evaluation

In [ ]:
##########################
# INSERT ACC VALUES IN % #
##########################

attack_points = lambda x: (1 / (1 + np.exp((-x / 3) + 21))) * 40
training_points = lambda x: (1 / (1 + np.exp((-x / 8) + 7))) * 40

# Plot sigmoids
from matplotlib import pyplot as plt
x = np.linspace(0, 110, 100)
figure = plt.figure(figsize=(15, 4))
figure.add_subplot(1, 2, 1)
plt.title("Training Accuracy\nPoints")
plt.plot(x, training_points(x))
plt.plot(train_acc, training_points(train_acc), 'ro')
plt.grid(True)
figure.add_subplot(1, 2, 2)
plt.title("MIA Accuracy\nPoints")
plt.plot(x, attack_points(x))
plt.plot(attack_acc, attack_points(attack_acc), 'ro')
plt.grid(True)
plt.show()

points = training_points(train_acc) - attack_points(attack_acc)
print("Points: {:.1f}".format(points/10))